In [ ]:

import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv
import re

In [2]:
load_dotenv(override=True)
api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

In [3]:
envio_para = os.getenv('ENVIO_PARA')

In [4]:
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME").strip()
email_senha = os.getenv("EMAIL_PASSWORD").strip()
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG').strip()
destinatario_rocio = os.getenv('DESTINATARIO_ROCIO').strip()
destinatario_scs = os.getenv('DESTINATARIO_SCS').strip()
destinatario_iir = os.getenv('DESTINATARIO_IIR').strip()
destinatario_vivi = os.getenv('DESTINATARIO_VIVIANE').strip()
destinatario_capibaribe= os.getenv('DESTINATARIO_CAPIBERIBE').strip()

In [5]:
auth_url = urljoin(api_url, "/sessions")
body = {
    "nome": api_username,
    "password":api_password
}

response = requests.post(auth_url, json=body)
token = response.json()["token"]
auth_token = "Bearer " + token

# Requisição para obter os dados do Pessoas
pessoas_url = urljoin(api_url, "/pessoas?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(pessoas_url, headers=headers)
pessoas = response.json()
pessoas = pd.DataFrame(pessoas)

pessoas.head(5)

id                                     ds_nome  co_tipo_gn co_equipe_gn  \
0  192                              Emerson Aredes       704.0         None   
1  196                               Kleber Giungi       562.0         None   
2  198                          Suporte Polo Trial       664.0         None   
3  199  Dra. Caroline Cândida Carvalho de Oliveira       387.0         None   
4  200                       Dra. Tânia Caltabiano       387.0         None   

   co_formacao_maior_nivel_gn cv_ingles cv_portugues  \
0                        31.0      None         None   
1                         NaN      None         None   
2                         NaN      None         None   
3                        30.0      None         None   
4                        29.0      None         None   

  dt_ultimo_certificado_gcp registro_profissional  \
0                      None                         
1                      None                         
2                      None                  None   
3                2020-09-28            202.285 SP   
4                      None                         

  declaracao_confidencialidade  ... equipe_padrao  equipe_padrao_kits  \
0                         None  ...             1                 0.0   
1                         None  ...             1                 1.0   
2                         None  ...             0                 0.0   
3                         None  ...             1                 0.0   
4                         None  ...             0                 0.0   

   equipe_padrao_farmacia  equipe_padrao_naocega  co_externo  \
0                     0.0                    0.0        None   
1                     1.0                    0.0        None   
2                     0.0                    0.0        None   
3                     0.0                    0.0        None   
4                     0.0                    0.0        None   

                                    dados_co_tipo_gn dados_co_equipe_gn  \
0          {'id': 704, 'ds_descricao': 'Suporte TI'}               None   
1  {'id': 562, 'ds_descricao': 'Equipe padrão (ad...               None   
2             {'id': 664, 'ds_descricao': 'Suporte'}               None   
3  {'id': 387, 'ds_descricao': 'Pesquisador Princ...               None   
4  {'id': 387, 'ds_descricao': 'Pesquisador Princ...               None   

                  dados_co_formacao_maior_nivel_gn  \
0           {'id': 31, 'ds_descricao': 'Mestrado'}   
1                                             None   
2                                             None   
3     {'id': 30, 'ds_descricao': 'Especialização'}   
4  {'id': 29, 'ds_descricao': 'Superior completo'}   

                             dados_centro  \
0                                    None   
1                                    None   
2                                    None   
3  {'id': 3, 'descricao': 'Leforte HMCG'}   
4  {'id': 3, 'descricao': 'Leforte HMCG'}   

                          dados_pessoa_especialidade  
0                                                 []  
1                                                 []  
2                                                 []  
3  [{'id': 3, 'co_pessoa': 199, 'co_pessoa_especi...  
4  [{'id': 4, 'co_pessoa': 200, 'co_pessoa_especi...  

[5 rows x 22 columns]

In [6]:
protocolo_url = urljoin(api_url, "/protocolo?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(protocolo_url, headers=headers)
protocolo = response.json()
protocolo = pd.DataFrame(protocolo)
protocolo.head(2)

id                                   titulo_protocolo cor_agenda  \
0   4  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   
1   5  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   

  numero_protocolo apelido_protocolo  coordenador     pi  pesquisador_backup  \
0    BTK-COV-202BR               BTK        261.0  306.0               210.0   
1    BTK-COV-202BR               BTK        310.0  199.0               227.0   

   co_pessoa_regulatorio             data_cadastro  ...  \
0                  214.0  2021-04-05T18:37:50.000Z  ...   
1                  214.0  2021-07-07T12:28:33.000Z  ...   

                 dados_aprovacao_cep  \
0  {'id': 60, 'ds_descricao': 'Sim'}   
1  {'id': 60, 'ds_descricao': 'Sim'}   

                               dados_co_centro  \
0       {'id': 3, 'descricao': 'Leforte HMCG'}   
1  {'id': 2, 'descricao': 'Leforte Liberdade'}   

                                            PessoaPI  \
0  {'id': 306, 'ds_nome': 'Dr. Carlos Augusto Qua...   
1  {'id': 199, 'ds_nome': 'Dra. Caroline Cândida ...   

                                   PessoaCoordenador  \
0            {'id': 261, 'ds_nome': 'Giovana Rosas'}   
1  {'id': 310, 'ds_nome': 'Isabel Cristina dos Sa...   

                                   PessoaPesquisador  \
0                                               None   
1  {'id': 227, 'ds_nome': 'Viviane Santana da Sil...   

                                   PessoaRegulatorio  \
0  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   
1  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   

                             fase_pesquisa  \
0  {'id': 123, 'ds_descricao': 'Fase III'}   
1   {'id': 122, 'ds_descricao': 'Fase II'}   

                                     status  \
0  {'id': 296, 'ds_descricao': 'Concluído'}   
1  {'id': 296, 'ds_descricao': 'Concluído'}   

                               tipo_iniciativa  \
0  {'id': 506, 'ds_descricao': 'Patrocinador'}   
1  {'id': 506, 'ds_descricao': 'Patrocinador'}   

                                   nome_patrocinador  
0  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  
1  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  

[2 rows x 124 columns]

In [7]:
css_hover = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-family: Arial, sans-serif;
        font-size: 14px;
        border-radius: 8px;
        overflow: hidden; /* Para manter o arredondamento no hover */
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    th, td {
        padding: 10px 14px;
        text-align: left;
        border-bottom: 1px solid #ddd;
    }
    th {
        background-color: #007bff;
        color: white;
        font-weight: 600;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
    tr:hover {
        background-color: #eaf3ff;
        transition: background-color 0.2s ease-in-out;
    }
</style>
"""

In [8]:
centros = protocolo[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros

id   apelido_protocolo numero_protocolo     co_externo  \
0        4                 BTK    BTK-COV-202BR  BTK-COV-202BR   
1        5                 BTK    BTK-COV-202BR  BTK-COV-202BR   
2        8         RIGEL-FOCUS     C-935788-061   C-935788-061   
3        9         BIOTEST-998     ESsCOVID-998           None   
4       10              GARDEN       ACT-CS-006     ACT-CS-006   
...    ...                 ...              ...            ...   
2336  2547   Chron e obesidade                            None   
2337  2548   Chron e obesidade                            None   
2338  2549  Transplante de Rim                            None   
2339  2550      R9933-PAD-2394                            None   
2340  2551         Oxandrolona                            None   

                                         apelido_centro  
0                                    BTK - Leforte HMCG  
1                               BTK - Leforte Liberdade  
2                       RIGEL-FOCUS - Leforte Liberdade  
3                            BIOTEST-998 - Leforte HMCG  
4                                 GARDEN - Leforte HMCG  
...                                                 ...  
2336             Chron e obesidade - Clínica CardialMed  
2337           Chron e obesidade - Santa Casa de Santos  
2338  Transplante de Rim - Maternidade e Cirurgia No...  
2339                      R9933-PAD-2394 - Leforte HMCG  
2340                  Oxandrolona - Centro Vila Olimpia  

[2341 rows x 5 columns]

# VENCIMENTO DE GCP (MODELAGEM E FUNÇÕES DE ENVIO)

In [9]:
gcp = pessoas[['dados_centro', 'dt_ultimo_certificado_gcp', 'ds_nome','dados_co_tipo_gn']].copy()
gcp.loc[:, 'dados_centro_id'] = gcp['dados_centro'].apply(lambda x: x['id'] if x is not None else None)
gcp.loc[:, 'dados_centro_descricao'] = gcp['dados_centro'].apply(lambda x: x['descricao'] if x is not None else None)
gcp.loc[:, 'tipo_gn'] = gcp['dados_co_tipo_gn'].apply(lambda x: x['ds_descricao'] if x is not None else None)

gcp_modelado = gcp.drop(['dados_centro', 'dados_centro_id', 'dados_co_tipo_gn'], axis=1)
gcp_modelado.head(5)

dt_ultimo_certificado_gcp                                     ds_nome  \
0                      None                              Emerson Aredes   
1                      None                               Kleber Giungi   
2                      None                          Suporte Polo Trial   
3                2020-09-28  Dra. Caroline Cândida Carvalho de Oliveira   
4                      None                       Dra. Tânia Caltabiano   

  dados_centro_descricao                                            tipo_gn  
0                   None                                         Suporte TI  
1                   None  Equipe padrão (adicionada automaticamente para...  
2                   None                                            Suporte  
3           Leforte HMCG                              Pesquisador Principal  
4           Leforte HMCG                              Pesquisador Principal

In [10]:
def verificar_vencimento_contratos(df_contratos, nome_centro, dias_para_vencimento=30):

    df_contratos['Assinatura do GCP'] = pd.to_datetime(df_contratos['Assinatura do GCP'], errors='coerce')

    hoje = datetime.today()
    limite_vencimento = hoje + timedelta(days=dias_para_vencimento)

    df_contratos['data_vencimento'] = df_contratos['Assinatura do GCP'] + timedelta(days=2 * 365)

    filtro_vencendo = (df_contratos['data_vencimento'] > hoje) & (df_contratos['data_vencimento'] <= limite_vencimento)

    filtro_sem_data = df_contratos['Assinatura do GCP'].isna()

    contratos_relevantes = df_contratos[filtro_vencendo | filtro_sem_data]

    if contratos_relevantes.empty:
        print(f"🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}.")
    else:
        print(f"⚠️ Contratos GCP sem data registrada no sistema e/ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}:")
        print(contratos_relevantes[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']])

    return contratos_relevantes

In [11]:
def filtrar_centros(df, lista_centros):

    filtro = df['dados_centro_descricao'].isin(lista_centros)
    centro_filtrado = df[filtro]
    resultado = centro_filtrado[centro_filtrado['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
    
    resultado = resultado.rename(columns={
        'dt_ultimo_certificado_gcp': 'Assinatura do GCP',
        'ds_nome': 'Nome',
        'dados_centro_descricao': 'Centro',
        'tipo_gn': 'Função'
    })
    
    nova_ordem = ['Centro', 'Nome', 'Função', 'Assinatura do GCP']
    return resultado[nova_ordem]


# Agora você pode chamar a função para cada grupo de centros:
hmcg = ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 'Leforte Liberdade', 'Clinica CardialMed']
venc_gcp_hmcg = filtrar_centros(gcp_modelado, hmcg)

rocio = ['Maternidade e Cirurgia Nossa Senhora do Rocio SA']
venc_gcp_rocio = filtrar_centros(gcp_modelado, rocio)

iir_coord = ['Hospital das Clínicas de Itajubá', 'Saint-Beaute Clinique', 'Hospital Salvalus','Consultório Lopes e Sartorelli', ' Clínica Berger','QualiVida Higienópolis','Endolap Saúde',' Associação Brasileira de Esclerose Múltipla','Centro Vila Olimpia']
venc_gcp_iir_coord = filtrar_centros(gcp_modelado, iir_coord)

envio_viviane = ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirã Preto',
    'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos Respiratórios',
    'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
    'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara',
    'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
    'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
    'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
    'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul']
venc_gcp_envio_viviane = filtrar_centros(gcp_modelado, envio_viviane)

stacasa_santos = ['Santa Casa de Santos']
venc_gcp_stacasa_santos = filtrar_centros(gcp_modelado, stacasa_santos)

capibaribe = ['Hospital do Capibaribe']
venc_gcp_capibaribe = filtrar_centros(gcp_modelado, capibaribe)


In [ ]:
contratos_hmcg = verificar_vencimento_contratos(venc_gcp_hmcg, "HMCG", dias_para_vencimento=30)
contratos_rocio = verificar_vencimento_contratos(venc_gcp_rocio, "Rocío", dias_para_vencimento=30)
contratos_iir_coord = verificar_vencimento_contratos(venc_gcp_iir_coord, "IRR COORD", dias_para_vencimento=30)
contratos_vivi = verificar_vencimento_contratos(venc_gcp_envio_viviane, "Envio Viviane", dias_para_vencimento=30)
contratos_stacsantos = verificar_vencimento_contratos(venc_gcp_stacasa_santos, "Santa Casa de Santos", dias_para_vencimento=30)
contratos_capiberibe = verificar_vencimento_contratos(venc_gcp_capibaribe, "Capiberibe", dias_para_vencimento=30)

In [ ]:
def enviar_email(destinatário,df_tratado):
    try:
        if df_tratado is None or df_tratado.empty:
            print("Não há contratos vencidos para enviar por e-mail.")
            return

        tabela_html = df_tratado[['Centro', 'Nome', 'Assinatura do GCP', 'data_vencimento']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Bcc'] = ', '.join(destinatário)
        msg['Subject'] = "GCP com vencimento nos próximos 30 dias ou sem data informada - "

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>Vencimento de GCP</h2>
                <p>Olá,</p> 
                <br>
                <p>Segue abaixo GCP que vencerão no próximo mês ou sem data informada no sistema:</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridadas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port
    ) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            server.sendmail(msg)

        print("E-mail de GCP que já venceram enviado com sucesso!")
    
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

In [ ]:
enviar_email([destinatario_hmcg], contratos_hmcg)
enviar_email([destinatario_rocio], contratos_rocio)
enviar_email([destinatario_iir], contratos_iir_coord)
enviar_email([destinatario_scs], contratos_stacsantos)
enviar_email([destinatario_vivi], contratos_vivi)
enviar_email([destinatario_capibaribe], contratos_capiberibe)

# VISITAS DE SEGUIMENTO (MODELAGEM E FUNÇÕES DE ENVIO)

In [ ]:
participantes_visita_url = urljoin(api_url, "/participante_visita?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(participantes_visita_url, headers=headers)
participantes_visita = response.json()
participantes_visita = pd.DataFrame(participantes_visita)

participantes_visita.head(2)

In [ ]:
seguimento = participantes_visita[['dados_participante', 'data_estimada', 'dados_status', 'dados_visita']].copy()

seguimento.loc[:, 'dados_participante_id'] = seguimento['dados_participante'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'id_participante'] = seguimento['dados_participante'].apply(lambda x: x['id_participante'] if x is not None else None)
seguimento.loc[:, 'dados_protocolo'] = seguimento['dados_participante'].apply(lambda x: x['dados_protocolo'] if x is not None else None)

seguimento.loc[:, 'dados_status_id'] = seguimento['dados_status'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'ds_descricao'] = seguimento['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
seguimento.loc[:, 'ds_nome_visita'] = seguimento['dados_visita'].apply(lambda x: x['ds_nome_visita'] if x is not None else None)

seguimento.loc[:, 'apelido_protocolo'] = seguimento['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)
seguimento['data_estimada'] = pd.to_datetime(seguimento['data_estimada']).dt.date
seguimento.head(2)

In [ ]:
seguimento_tratado = seguimento.drop(['dados_participante','dados_status','id_participante','dados_status_id'], axis=1)
seguimento_tratado.loc[:, 'id'] = seguimento['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
seguimento_tratado.head(2)

In [ ]:
seguimentos = pd.merge(centros, seguimento_tratado, on='id', how='inner')
seguimentos

In [ ]:
seguimentos['apelido_protocolo_x'] = seguimentos['apelido_protocolo_x'].str.strip()
visitas_filtrado = seguimentos[seguimentos['ds_descricao'].str.contains('Pendente')]
nova_ordem = ["apelido_centro", "dados_participante_id", "ds_nome_visita","data_estimada","ds_descricao"]
visitas_reordenado = visitas_filtrado[nova_ordem]
visitas_reordenado.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_participante_id': 'ID Participante', 'ds_nome_visita': 'Tipo Visita',
                          'data_estimada':'Data Estimada', 'ds_descricao':'Status'}, inplace=True)
visitas_reordenado

In [ ]:
datas_visitas = visitas_reordenado.dropna(subset=["Data Estimada"])
datas_visitas = datas_visitas.sort_values(by= "Data Estimada", ascending=True)
datas_visitas = datas_visitas[~datas_visitas["Tipo Visita"].isin(["Unscheduled", "Triagem", "End of Study"])]
valores_contagem = datas_visitas['Tipo Visita'].value_counts()
datas_visitas

In [ ]:
def verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20):
    datas_visitas['Data Estimada'] = pd.to_datetime(datas_visitas['Data Estimada'])
    
    hoje = datetime.today()
    limite_visita = hoje + timedelta(days=dias_para_visita)
    
    visitas_futuras = datas_visitas[
        (datas_visitas['Data Estimada'] >= hoje) & 
        (datas_visitas['Data Estimada'] <= limite_visita)
    ]
    
    if visitas_futuras.empty:
        print(f"Não há visitas programadas para os próximos {dias_para_visita} dias.")
    else:
        print(f"Visitas programadas para os próximos {dias_para_visita} dias:")
        print(visitas_futuras[['Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada','Status']])
    
    return visitas_futuras

visitas_20_dias = verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20)
# próximas_visitas = visitas_20_dias.drop(['dados_visita'], axis=1)
# próximas_visitas.head(2)

In [ ]:
def enviar_emails(próximas_visitas):
    global enviar_para  # Acessa a variável global 'enviar_para'
    
    try:
        if próximas_visitas is None or próximas_visitas.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        colunas_esperadas = {'Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada', 'Status'}
        if not colunas_esperadas.issubset(próximas_visitas.columns):
            print("Erro: DataFrame não contém todas as colunas esperadas.")
            return

        if not all([smtp_server, email_usuario, email_senha, email_port]):
            print("Erro: Configurações de e-mail estão incompletas.")
            return

        # Validação de e-mails com regex
        email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        if isinstance(enviar_para, list):
            enviar_para = [email.strip() for email in enviar_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            enviar_para = []

        # Criação da tabela HTML
        tabela_html = próximas_visitas[list(colunas_esperadas)].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        # Montagem da mensagem
        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Seguimento - próximos 20 dias"
        
        # Define o campo BCC apenas se houver destinatários válidos
        if enviar_para:
            msg['Bcc'] = ', '.join(enviar_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de seguimento programadas para os próximos 20 dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        # Envio de e-mail
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            
            # Lista final de destinatários
            destinatarios = [email_usuario] + enviar_para
            if destinatarios:
                server.sendmail(email_usuario, destinatarios, msg.as_string())
                print("E-mail de visitas pendentes enviado com sucesso!")
            else:
                print("Nenhum destinatário válido. E-mail não enviado.")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(visitas_20_dias)

# VISITAS DE MONITORIA (MODELAGEM E FUNÇÃO ENVIO DE EMAIL)

In [ ]:
agenda_url = urljoin(api_url, "/agenda?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(agenda_url, headers=headers)
agenda = response.json()
agenda = pd.DataFrame(agenda)

agenda.head(2)

In [ ]:
monitoria = agenda[['dados_protocolo', 'dados_tipo', 'dados_status', 'data_estimada_filter']].copy()

monitoria.loc[:, 'dados_protocolo_id'] = monitoria['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'apelido_protocolo'] = monitoria['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)

monitoria.loc[:, 'dados_tipo_id'] = monitoria['dados_tipo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'dados_tipo_tipo'] = monitoria['dados_tipo'].apply(lambda x: x['tipo'] if x is not None else None)

monitoria.loc[:, 'dados_status_id'] = monitoria['dados_status'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'ds_descricao'] = monitoria['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

monitoria = monitoria.drop(['dados_protocolo','dados_status', 'dados_tipo_id','dados_status_id'], axis=1)
monitoria.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)

monitoria.head(2)

In [ ]:
monitorias = pd.merge(centros, monitoria, on='id', how='inner')
monitorias

In [ ]:
monitorias_filtrado = monitorias[monitorias['dados_tipo_tipo'].str.contains('Monitoria')]
monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()
monitorias_filtrado

In [ ]:
def verificar_monitorias_pendentes():
    monitorias_filtrado.loc[:, 'data_estimada_filter'] = pd.to_datetime(monitorias_filtrado['data_estimada_filter'])
    hoje = datetime.today()

    monitorias_pendentes = monitorias_filtrado[
        (monitorias_filtrado['ds_descricao'] == 'Pendente') &
        (monitorias_filtrado['data_estimada_filter'] >= hoje)
    ]
    monitorias_pendentes = monitorias_pendentes.sort_values(by='data_estimada_filter', ascending=True)

    print(monitorias_pendentes[['dados_tipo_tipo', 'data_estimada_filter', 'apelido_protocolo_x', 'ds_descricao','apelido_centro']])
    return monitorias_pendentes

schedule.every().monday.at("09:00").do(verificar_monitorias_pendentes)

monitorias_pendentes_df = verificar_monitorias_pendentes()

In [ ]:
monitorias_pendentes_df.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_tipo_tipo': 'Monitoria', 'data_estimada_filter': 'Data Estimada',
                                        'ds_descricao': 'Status'}, inplace=True)
monitorias_pendentes_df

In [ ]:
def enviar_emails(monitorias_pendentes_df):
    global envio_para
    try:
        if monitorias_pendentes_df is None or monitorias_pendentes_df.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        tabela_html = monitorias_pendentes_df[['Estudo/Centro', 'Monitoria', 'Data Estimada', 'Status']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Monitoria Pendentes"

        # Validação da lista de destinatários
        if isinstance(envio_para, list):
            # Expressão regex para validar e-mails
            email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
            envio_para = [email.strip() for email in envio_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            envio_para = []

        # Verifica se há destinatários antes de enviar
        destinatarios = [email_usuario] + envio_para
        if not destinatarios:
            print("Nenhum destinatário válido encontrado. O e-mail não foi enviado.")
            return

        # Somente adiciona Bcc se houver destinatários válidos
        if envio_para:
            msg['Bcc'] = ', '.join(envio_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de monitoria programadas para os pr&oacute;ximos dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            # Envio corrigido para passar apenas destinatários válidos
            server.sendmail(email_usuario, destinatarios, msg.as_string())

        print("E-mail de visitas pendentes enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(monitorias_pendentes_df)